## Augsbrug OParl Analyse

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser
import time

# 1. Konfiguration: Der echte Endpunkt für Augsburg
# Viele Städte nutzen 'ris-muenchen.de' oder ähnliche Software-Provider.
# Für Augsburg ist der Einstiegspunkt (System URL) oft:
OPARL_BASE_URL = "https://ratsinfo-augsburg.de/oparl/v1"

def fetch_meetings(limit_pages=5):
    """
    Holt Sitzungen von der API.
    Limit_pages ist wichtig, um den Server beim Testen nicht zu überlasten.
    """
    url = f"{OPARL_BASE_URL}/meeting"
    meetings_data = []
    page_count = 0

    print(f"Starte Abfrage von: {url}")

    while url and page_count < limit_pages:
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            # Die eigentlichen Daten liegen oft im 'data'-Array (je nach OParl Version)
            # Die Spec sagt: Die Antwort ist eine Liste oder ein Objekt mit 'data'
            items = data.get('data', []) if isinstance(data, dict) else data

            for item in items:
                meetings_data.append({
                    'name': item.get('name', 'Unbenannte Sitzung'),
                    'start': item.get('start'),
                    'end': item.get('end'), # Das kritische Feld
                    'organization': item.get('organization', ['Unknown'])[0] # Vereinfachung
                })

            # Pagination: Suche den Link zur nächsten Seite
            links = data.get('links', {})
            url = links.get('next')

            page_count += 1
            print(f"Seite {page_count} geladen... ({len(items)} Sitzungen)")

            # Höflichkeitspause für den Server
            time.sleep(0.2)

        except Exception as e:
            print(f"Fehler beim Abrufen: {e}")
            break

    return pd.DataFrame(meetings_data)

# 2. Datenanalyse & Bereinigung
df = fetch_meetings(limit_pages=10) # Hole ca. 10 Seiten an Daten

# Konvertiere Strings in echte Datetime-Objekte
df['start'] = pd.to_datetime(df['start'], utc=True)
df['end'] = pd.to_datetime(df['end'], utc=True)

# Feature Engineering: Wochentag und Stunde extrahieren
# Wir konvertieren nach 'Europe/Berlin', damit 17:00 auch 17:00 ist
df['start_local'] = df['start'].dt.convert_timezone('Europe/Berlin')
df['weekday'] = df['start_local'].dt.day_name()
df['hour'] = df['start_local'].dt.hour

# Reliability Check: Filtern von ungültigen Daten (z.B. Startdatum fehlt)
df_clean = df.dropna(subset=['start'])

print(f"\nAnalyse basierend auf {len(df_clean)} Sitzungen.")

# 3. Visualisierung (Stufe 1: Der Überblick)
# Wann finden Sitzungen statt?
plt.figure(figsize=(10, 6))
counts = df_clean['hour'].value_counts().sort_index()
counts.plot(kind='bar', color='#A020F0', edgecolor='black') # Augsburg-Farbe wäre eigentlich Zirbelnuss-Grün/Rot

plt.title('Der Biorhythmus des Augsburger Stadtrats', fontsize=14)
plt.xlabel('Uhrzeit (Stunde des Startbeginns)', fontsize=12)
plt.ylabel('Anzahl der Sitzungen', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

# 4. Kleine Text-Ausgabe für den "Aha-Effekt"
latest_meeting = df_clean.loc[df_clean['hour'] >= 20]
if not latest_meeting.empty:
    print("\n--- Nachteulen-Alarm ---")
    print("Beispiele für späte Sitzungen (Start nach 20 Uhr):")
    print(latest_meeting[['name', 'start_local']].head(3))